## Preparing the BLAST databases

Two databases were obtained:

- All annotated 16S rRNA from the NCBI ftp [website](https://ftp-ncbi-nlm-nih-gov.ejournal.mahidol.ac.th/blast/db/v5/)
- Annotated 16S rRNA used by the tool [MicFunPred](https://github.com/microDM/MicFunPred)

Because these databases were of incompatible versions (the MicFun is a version 4 database while the NCBI is version 5), I extracted all the

# Combining nucleotide databases
blastdb_aliastool -dblist "db1 db2" -dbtype nucl -out 16sCombined -title "16sCombined"

In [ ]:
# Verify the combined database
blastdbcmd -entry all -db 16sCombined -out combined_check.fasta

In [ ]:
# Search with a specific output format
# -outfmt 5 is for xml format, 6 for tabular
blastn -query <query file in fasta or raw seqs> -db 16sCombined -outfmt 5 -out <output filenme>

## Parsing BLAST output

Will be using the Biopython package for this.


In [1]:
import subprocess as sp
import numpy as np
from Bio import SearchIO, SeqIO
import pandas as pd
from Bio.Blast import NCBIXML
blastdb: str = "data/blastdb/16sCombined"
params: str = (
            "-outfmt 5 " +
            "-max_target_seqs 10 " +
            "-perc_identity 90 "
)
to_parse = "testblast.fasta"
parse_type = "fasta"

seq_frame = pd.DataFrame()
sequences: np.array = np.array([])
ids: np.array = np.array([])
top_hits: np.array = np.array([])
# Search the fasta file
for query in SeqIO.parse(to_parse, parse_type):
    temp = open('temp.fasta', 'w+')
    temp.write(f'>{query.id}\n{query.seq}')
    t = temp.read()
    temp.close
    blastsearch: str = f"blastn {params} -query temp.fasta -db {blastdb} > temp.txt"
    sp.run(blastsearch, shell=True)
    parse = SearchIO.read('temp.txt', 'blast-xml')
    if not len(parse):
        continue
    sequences = np.append(sequences, str(query.seq))
    top_hits = np.append(top_hits, parse[0].id)
    ids = np.append(ids, query.id)

# Construct the dataframe
seq_frame['Query id'] = ids
seq_frame['Query sequence'] = sequences
seq_frame['Top hit'] = top_hits
seq_frame.to_csv('output.csv')

/home/sc31/Bio_SDD/miniconda3/lib/python3.10/site-packages/Bio/SearchIO/_legacy/__init__.py:12: BiopythonDeprecationWarning: The 'Bio.SearchIO._legacy' module for parsing BLAST plain text output is deprecated and will be removed in a future release of Biopython. Consider generating your BLAST output for parsing as XML or tabular format instead.
  warnings.warn(
/home/sc31/Bio_SDD/miniconda3/lib/python3.10/site-packages/Bio/SearchIO/BlastIO/blast_xml.py:339: BiopythonParserWarning: Renaming hit ID '2663763413_Escherichia_coli_TOP293-4' to a BLAST-generated ID 'gnl|BL_ORD_ID|27423' since the ID was already matched by your query 'M01457:82:000000000-B6GV3:1:1104:19070:5914'. Your BLAST database may contain duplicate entries.
  warnings.warn(
/home/sc31/Bio_SDD/miniconda3/lib/python3.10/site-packages/Bio/SearchIO/BlastIO/blast_xml.py:339: BiopythonParserWarning: Renaming hit ID '2660237930_Escherichia_coli_TOP550-1' to a BLAST-generated ID 'gnl|BL_ORD_ID|27419' since the ID was already mat

In [5]:
import pandas as pd
out = pd.read_csv('output.csv')

(1108,)

In [21]:
# For parsing through blast output with 1+ queries
result_handle = open("my_blast.xml")
blast_records = NCBIXML.parse() # Is an iterator object, so you can loop over it to access each query
for record in blast_records:

SyntaxError: incomplete input (2602383775.py, line 4)

In [5]:
import pandas as pd
new = pd.read_csv('data/blastdb/tab.txt', sep='\t')
new

,M01457:82:000000000-B6GV3:1:1101:20647:1278,JX224393.1.1496_Desulfotomaculum,75.745,235,52,3,6,240,340,569,1.60e-24,113
0,M01457:82:000000000-B6GV3:1:1101:20647:1278,JX225284.1.1296_Desulfotomaculum,87.778,90,11,0,6,95,123,212,2.680000e-22,106
1,M01457:82:000000000-B6GV3:1:1101:20647:1278,JX224226.1.1352_Desulfotomaculum,92.105,76,4,2,6,80,339,413,2.680000e-22,106
2,M01457:82:000000000-B6GV3:1:1101:20647:1278,JX224392.1.1496_Desulfotomaculum,85.149,101,15,0,6,106,340,440,9.650000e-22,104
3,M01457:82:000000000-B6GV3:1:1101:20647:1278,JX224446.1.1287_Desulfosporosinus,90.789,76,5,2,6,80,136,210,1.250000e-20,100


# Datasets

These data sets will be the test datasets that you use

- https://www.ebi.ac.uk/ena/browser/view/PRJEB9039 - Microbial life in geothermal and volcanic areas
- https://www.ebi.ac.uk/ena/browser/view/PRJEB10824 - Functional diversity of Outokumpu fractures
- https://www.ebi.ac.uk/ena/browser/view/PRJEB10822 Outokumpu fracture communities
- https://www.ebi.ac.uk/ena/browser/view/PRJNA188465 Sulfidic cave snottites Metagenome
- https://www.ebi.ac.uk/ena/browser/view/PRJNA487671 BACTERIAL COMMUNITIES IN THE ACID AND THERMOPHILIC CRATER-LAKE OF THE VOLCANO "EL CHICHON", MEXICO
- https://www.ebi.ac.uk/ena/browser/view/PRJEB36834 Prokaryotic Community Profiling of the Mt. Makiling Mudspring
- https://www.ebi.ac.uk/ena/browser/view/PRJEB10725 This dataset has both amplicon and shotgun data, so you would try out your method on the amplicon data only and see if it was able to recover the same go annotations. But it is 551 gb so be careful when downloading. Could use this as a training set
  - https://www.ebi.ac.uk/ena/browser/view/PRJEB87662 This too
  - This could also be used, only 14 gb https://www.ebi.ac.uk/ena/browser/view/PRJNA394849
  - https://www.ebi.ac.uk/ena/browser/view/PRJEB34718 This one is on lichens, probably the best one because of its size
- [Here](https://www-ncbi-nlm-nih-gov.ejournal.mahidol.ac.th/sra) is a link to the 16srRNA from [this](https://www.ebi.ac.uk/ena/browser/view/PRJNA682552?show=publications) sediment study
- [Another](https://www-ncbi-nlm-nih-gov.ejournal.mahidol.ac.th/sra) 16srRNA resource from [this](https://www.ebi.ac.uk/metagenomics/studies/MGYS00000666#overview) soil study


# Machine learning

https://www.kaggle.com/code/singhakash/dna-sequencing-with-machine-learning What you should use for the learning


# Metagenomic processing
Most of this will be handled with qiime2, a cli program
- Presentation outline
    - Introduce the topic and rationale
    - Data exploration 
        - characteristic noise in fastq files 
        - blast searches on overrepresented sequences 
    - Taxonomic classification
        - How you trained the models
    - Statistical methods for diversity
        - Alpha diversity 
        - Beta diversity 
            - ANCOM-BC
            - Differential abundance analysis
            - Permanova

# Working with MGnify API
Courtesy of https://shiny-portal.embl.de/shinyapps/app/06_mgnify-notebook-lab

In [3]:
import os
from IPython.display import Markdown

def get_variable_from_link_or_input(variable, name = 'accession', default = None):
    """
    Get a variable value, either from an ENV VAR that would have been set by the shiny_proxy_jlab_query_parms extension, or through direct user input.
    """
    var = os.getenv(variable)
    if var:
        display(Markdown(f'<span style="background-color: #0a5032; color: #fff; padding: 8px;">Using {name} <emph>{var}</emph> from the link you followed.</span>'))
    else:
        var = input(f'Type {"an" if name[0].lower() in "aeiou" else "a"} {name} [default: {str(default)}]')
    var = var or default
    print(f'Using "{var}" as {name}')
    return var

In [9]:
from jsonapi_client import Session
import pandas as pd

api_endpoint = 'biomes'


with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    view = map(lambda r: r.json, mgnify.iterate(api_endpoint))   
    view = pd.json_normalize(view) 
    display(view)

,type,id,attributes.samples-count,attributes.biome-name,attributes.lineage
0,biomes,root:Engineered,284,Engineered,root:Engineered
1,biomes,root:Engineered:Biogas plant,126,Biogas plant,root:Engineered:Biogas plant
2,biomes,root:Engineered:Biogas plant:Wet fermentation,111,Wet fermentation,root:Engineered:Biogas plant:Wet fermentation
3,biomes,root:Engineered:Bioreactor,1003,Bioreactor,root:Engineered:Bioreactor
4,biomes,root:Engineered:Bioreactor:Continuous culture,648,Continuous culture,root:Engineered:Bioreactor:Continuous culture
...,...,...,...,...,...
486,biomes,root:Host-associated:Spiralia,4,Spiralia,root:Host-associated:Spiralia
487,biomes,root:Host-associated:Tunicates,0,Tunicates,root:Host-associated:Tunicates
488,biomes,root:Host-associated:Tunicates:Ascidians,0,Ascidians,root:Host-associated:Tunicates:Ascidians
489,biomes,root:Control,3,Control,root:Control


## Backup data
- Muztagh ata https://www.ncbi.nlm.nih.gov/bioproject/884890, single-end, illumina miniseq
- Barrow Alaska https://www.ebi.ac.uk/ena/browser/view/PRJEB9043, single-end, illumina hiseq 2000
- Devon ice cap https://www.ncbi.nlm.nih.gov/bioproject/564341, paired -end, illumina minseq
- Sverdrup Glacier https://www.ncbi.nlm.nih.gov/bioproject/899883, paired-end, illumina miniseq
- Greenland Ice sheet https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=517071